**Script formating the query result json of the ooa occupations query to the format needed for the [Apache ECharts sunburst visualisation](https://echarts.apache.org/examples/en/editor.html?c=sunburst-simple)**

Import libraries

In [ ]:
import requests, json

Fetch occupation-ontology query result JSON

In [ ]:
inJSON = requests.get("https://gams.uni-graz.at/archive/objects/query:ooa.occupations/methods/sdef:Query/getJSON").json()
#inJSON

Create result JSON

In [ ]:
### Create JSON for Apache ECharts ###
outJSON = []
for result in inJSON: # Lookup major groups
    count = result["count"]
    unit = result["HISCO_Unit"]
    unit_group = result["HISCO_Unit_group"]
    minor_group = result["HISCO_Minor_group"]
    major_group = result["HISCO_Major_group"]
    if not any(mag["name"] == major_group for mag in outJSON): # New major group
        outJSON.append({"name": major_group, "children": []})
    currentMag = next(mag for mag in outJSON if mag["name"] == major_group)
    if not any(mig["name"] == minor_group for mig in currentMag["children"]): # New minor group
        currentMag["children"].append({"name": minor_group, "children": []})
    currentMig = next(mig for mig in currentMag["children"] if mig["name"] == minor_group)
    if not any(ug["name"] == unit_group for ug in currentMig["children"]): # New unit group
        currentMig["children"].append({"name": unit_group, "children": []})
    currentUG = next(ug for ug in currentMig["children"] if ug["name"] == unit_group)
    currentUG["children"].append({"name": unit, "value": int(count)})
print(json.dumps(outJSON, indent=2))

Write JSON to file

In [ ]:
# with open("eCharts.json", "w", encoding="utf-8") as file:
#     file.write(json.dumps(outJSON, indent=2, ensure_ascii=False))